# Test Case LVV-t1992
This case will verify that the M2/Camera Hexapod is minimally functional.
The blocks below represent different steps of the test case.

Requirements
* EFD
* Hexapod(s) powered on
* Thermal sensors attached to the six actuators of the hexapod.
* CSC running

This test will require manual verification of certain events and telemetry in the summit EFD.
Also manual verification of appropriate temperatures for each actuator.

In [1]:
from lsst.ts import salobj
from lsst.ts.idl.enums import MTHexapod
import logging
import asyncio
import os

/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-0.4.1/lib/python3.8/site-packages/jose/backends/cryptography_backend.py:23: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes, int_to_bytes


The following block sets the necessary environment variables for setting up the DDS/SAL communication

In [ ]:
#os.environ["LSST_DDS_DOMAIN"] = 'citest'
os.environ["LSST_DDS_HISTORYSYNC"] = "10"

In [2]:
STD_WAIT = 39 # This is the amount of time to wait in between a move just to make sure that the actuators are not overheating

In [3]:
log = logging.getLogger(__name__) # This sets up the logger for the test.

In [4]:
test_message = "M2 Hexapod Integration Test"
script = salobj.Controller("Script", index=42658885)

This is how you start the remote for the CSC.

In [5]:
domain = salobj.Domain()
print (domain)
#index=1 is the camera hexapod!! index=2 is the M2 hexapod!
hexapod_csc = salobj.Remote(name="MTHexapod", domain=domain, index=2)
print (hexapod_csc)
await hexapod_csc.start_task
#This is to bring the state machine in the right starting state
await salobj.set_summary_state(hexapod_csc, salobj.State.ENABLED)
#await salobj.set_summary_state(hexapod_csc, salobj.State.OFFLINE)

RuntimeError: Cannot get summaryState from MTHexapod

In [ ]:
def enabled_substate_callback(evt):
    """Print the enabled substate when event is received."""
    print(MTHexapod.EnabledSubstate(evt.enabledSubstate))

In [ ]:
def in_position_callback(evt):
    """Print the in position event when it is received."""
    print(evt.in_position)

This next block will make sure that the CSC is ready for the test

In [ ]:
connected = await hexapod_csc.evt_connected.aget() # This is how you get an event/telemetry with await remote.type_name.aget() type being [evt,tel] and name being the name of the topic
commandable = connected.command
telemetry_working = connected.telemetry

if not connected or not telemetry_working:
    raise Exception("Hexapod not connected or telemetry not being received.")

    #This is only for the real system. The simulater only has the AVAILABLE offline state 
'''   
controller_state = await hexapod_csc.evt_controllerState.aget()
state = controller_state.controllerState
offline_substate = controller_state.offlineSubstate
print(state)
if not state == salobj.State.OFFLINE and not offline_substate == MTHexapod.OfflineSubstate.AVAILABLE:
    raise Exception("Controller must be changed to Available Offline Substate.")
'''
commandable_by_dds = await hexapod_csc.evt_commandableByDDS.aget()
dds_state = commandable_by_dds.state
if not dds_state:
    raise Exception("Controller must in CommandableByDDS state.")


The next four blocks bring the CSC to the Enabled state.

In [ ]:
await hexapod_csc.cmd_enterControl.set_start() # Send the enterControl command notice the casing and the set_start method

In [ ]:
await hexapod_csc.cmd_start.set_start() 

In [ ]:
await hexapod_csc.cmd_enable.set_start()

In [ ]:
await hexapod_csc.cmd_clearError.set_start() # This clears the error

# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

In [ ]:
# Step skiped see deviation LVV-T1802 Test Step 5                                  
hexapod_csc.evt_controllerState.callback = enabled_substate_callback                                
await hexapod_csc.cmd_move.set_start(x=0,y=0,z=200, u=0,v=0,w=0,sync=True)



In [ ]:
script.log.info(f"START- {test_message} -- LVV-T1802 Test Step 7")
hexapod_csc.evt_controllerState.callback = enabled_substate_callback                                

await hexapod_csc.cmd_move.set_start(x=500,y=-500,z=200, u=0.01,v=-0.015,w=0,sync=True)
script.log.info(f"STOP- {test_message} -- LVV-T1802 Test Step 7")


In [ ]:
await asyncio.sleep(STD_WAIT)
# thermal sensors

# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

In [ ]:
# thermal sensors

This block moves the hexapod to a large position but stops it after waiting 3 seconds

In [ ]:
script.log.info(f"START- {test_message} -- LVV-T1802 Test Step 11")
await hexapod_csc.cmd_move.set_start(x=0,y=0,z=5000,u=0,v=0,w=0,sync=True)
await asyncio.sleep(3)
await hexapod_csc.cmd_stop.set_start()
script.log.info(f"STOP- {test_message} -- LVV-T1802 Test Step 11")

# cs = await hexapod_csc.evt_controllerState.aget()
# print(Hexapod.EnabledSubstate(cs.enabledSubstate))
# if cs.enabledSubstate != Hexapod.EnabledSubstate.CONTROLLED_STOPPING:
#     raise Exception(f"Controller's substate is not {Hexapod.EnabledSubstate.CONTROLLED_STOPPING}. It is {Hexapod.EnabledSubstate(cs.enabledSubstate)}")
# cs = await hexapod_csc.evt_controllerSubstate.aget()
# print(Hexapod.EnabledSubstate(cs.enabledSubstate))
# if cs.enabledSubstate != Hexapod.EnabledSubstate.STATIONARY:
#     raise Exception(f"Controller's substate is not {Hexapod.EnabledSubstate.STATIONARY}. It is {Hexapod.EnabledSubstate(cs.enabledSubstate)}")


In [ ]:
#Step 15
await asyncio.sleep(STD_WAIT)

# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

This block moves the hexapod via the lookup table:

In [ ]:
#This command is obsolete
#await hexapod_csc.cmd_moveLUT.set_start(az=180,elev=60, temp=10)
script.log.info(f"START- {test_message} -- LVV-T1802 Test Step 17")
await hexapod_csc.cmd_move.set_start(x=0,y=0,z=800,u=0,v=0,w=0,sync=True)

await hexapod_csc.cmd_setCompensationMode.set_start(enable=True)

await hexapod_csc.cmd_move.set_start(x=0,y=0,z=800,u=0,v=0,w=0,sync=True)

script.log.info(f"STOP- {test_message} -- LVV-T1802 Test Step 17")

In [ ]:
#Step 22
await asyncio.sleep(STD_WAIT)
# thermal sensors

# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

In [ ]:
script.log.info(f"START- {test_message} -- LVV-T1802 Test Step 24")
await hexapod_csc.cmd_move.set_start(x=500,y=800,z=200,u=0,v=0,w=0,sync=True)
await hexapod_csc.cmd_offset.set_start(x=0,y=0,z=500,u=0,v=0,w=0,sync=True)
script.log.info(f"STOP- {test_message} -- LVV-T1802 Test Step 24")


# cs = await hexapod_csc.evt_controllerState.aget()
# if cs.enabledSubstate != hexapod.SetEnabledSubstateParam.MOVE_POINT_TO_POINT:
#     raise Exception(f"Controller's substate is not {hexapod.SetEnabledSubstateParam.MOVE_POINT_TO_POINT}. It is {hexapod.SetEnabledSubstateParam(cs.enabledSubstate)}")
# cs = await hexapod_csc.evt_controllerSubstate.aget()
# if cs.enabledSubstate != hexapod.SetEnabledSubstateParam.STATIONARY:
#     raise Exception(f"Controller's substate is not {hexapod.SetEnabledSubstateParam.STATIONARY}. It is {hexapod.SetEnabledSubstateParam(cs.enabledSubstate)}")
# in_position = await hexapod_csc.evt_inPosition.aget(timeout=10)
# if in_position.inPosition is False:
#     raise Exception("Controller not in position.")

In [ ]:
#Step 27
await asyncio.sleep(STD_WAIT)

In [ ]:
await salobj.set_summary_state(hexapod_csc, salobj.State.STANDBY) # Transition the CSC to Standby state
await domain.close() # Close the remote connection

# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |